In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [4]:
stock = "CVX"
df = pd.DataFrame(yf.download(stock, '2010-01-01', '2022-12-31')["Close"])

[*********************100%***********************]  1 of 1 completed


In [39]:
def rolling(lag, data):
    A = []
    df = data.tolist()
    for i in range(lag, len(df)):
        A.append(df[i-lag:i]) 
    return pd.DataFrame(A)

def UoD(x):
    if x<0:
        return -1
    if x>0:
        return 1
    else: 
        return 0

def NN(x,y, input_dim, out_dim):
		model = Sequential()
		model.add(Dense(100, input_dim = input_dim, activation="sigmoid"))
		model.add(Dense(100, input_dim = 100, activation="sigmoid"))
		model.add(Dense(100, input_dim = 100, activation="sigmoid"))
		model.add(Dense(out_dim, input_dim = 5, activation="softmax"))

		optimizer=Adam()
		model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

		model.fit(x, y, epochs=1000, verbose=2)
		return model

In [19]:
df["returns"] =df["Close"].pct_change()
df["UoD"]= df["returns"].map(UoD)
df.dropna(inplace=True)

In [9]:
df["UoD"].shift(-3)[:-3]

Date
2010-01-05    1.0
2010-01-06    1.0
2010-01-07   -1.0
2010-01-08   -1.0
2010-01-11   -1.0
             ... 
2022-12-20    1.0
2022-12-21    1.0
2022-12-22   -1.0
2022-12-23    1.0
2022-12-27    1.0
Name: UoD, Length: 3268, dtype: float64

In [32]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
data = (scaler.fit_transform(df["Close"].values.reshape(-1,1)))
df["p"]=data

In [33]:
df

,Close,returns,UoD,p
Date,,,,
2010-01-06,79.629997,0.000126,1,0.189868
2010-01-07,79.330002,-0.003767,-1,0.187626
2010-01-08,79.470001,0.001765,1,0.188672
2010-01-11,80.879997,0.017742,1,0.199208
2010-01-12,80.410004,-0.005811,-1,0.195696
...,...,...,...,...
2022-12-23,177.399994,0.030916,1,0.920421
2022-12-27,179.630005,0.012571,1,0.937084
2022-12-28,176.979996,-0.014753,-1,0.917283


In [41]:
lag = 20
A = rolling(lag, df["p"])
b = df["UoD"].shift(-lag)[:-lag]


encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(b).reshape(-1, 1)).toarray()
deon, dete, keyon, keyte = train_test_split(A.values,y, test_size=1)

In [42]:
a = NN(deon,keyon,lag,3)

Epoch 1/1000
102/102 - 1s - loss: 0.7425 - accuracy: 0.5112 - 640ms/epoch - 6ms/step
Epoch 2/1000
102/102 - 0s - loss: 0.7215 - accuracy: 0.4971 - 290ms/epoch - 3ms/step
Epoch 3/1000
102/102 - 0s - loss: 0.7184 - accuracy: 0.5011 - 190ms/epoch - 2ms/step
Epoch 4/1000
102/102 - 0s - loss: 0.7163 - accuracy: 0.5091 - 183ms/epoch - 2ms/step
Epoch 5/1000
102/102 - 0s - loss: 0.7200 - accuracy: 0.5011 - 167ms/epoch - 2ms/step
Epoch 6/1000
102/102 - 0s - loss: 0.7183 - accuracy: 0.4891 - 153ms/epoch - 2ms/step
Epoch 7/1000
102/102 - 0s - loss: 0.7174 - accuracy: 0.5045 - 153ms/epoch - 2ms/step
Epoch 8/1000
102/102 - 0s - loss: 0.7221 - accuracy: 0.4995 - 157ms/epoch - 2ms/step
Epoch 9/1000
102/102 - 0s - loss: 0.7180 - accuracy: 0.4955 - 223ms/epoch - 2ms/step
Epoch 10/1000
102/102 - 0s - loss: 0.7210 - accuracy: 0.5029 - 241ms/epoch - 2ms/step
Epoch 11/1000
102/102 - 0s - loss: 0.7174 - accuracy: 0.5045 - 143ms/epoch - 1ms/step
Epoch 12/1000
102/102 - 0s - loss: 0.7157 - accuracy: 0.5192 - 

KeyboardInterrupt: 

In [14]:
d=a.predict(dete)
d

51/51 [==============================] - 0s 2ms/step


array([[0.50590086, 0.00244201, 0.49165714],
       [0.5059009 , 0.00244214, 0.49165696],
       [0.5058987 , 0.00244204, 0.49165922],
       ...,
       [0.5059011 , 0.00244218, 0.49165675],
       [0.5058996 , 0.00244185, 0.4916585 ],
       [0.50589937, 0.00244192, 0.4916587 ]], dtype=float32)